1.1 

Given the following information, write a python function that uses the Gordon Growth Model to estimate the price-to-book ratio for a company:

Required return on equity (r) = 0.12

Payout ratio (k) = 0.35

Return on equity (ROE) = 0.18

Book equity = $200 million

Note: Submit only the answer from your code, not your code. Answer with 2 decimal precision.

In [35]:
r = 0.12
k = 0.35
ROE = 0.18
book_equity = 200

g = (1 - k) * ROE

earnings = ROE * book_equity
price = earnings * k / (r - g)

price_to_book = price / book_equity
print("Price-to-book ratio:", round(price_to_book, 2))

Price-to-book ratio: 21.0


1.2 

Consider the following scenario:

A buy-side analyst works for an investment fund and has been tasked with picking stocks for the fund. They have been given the choice of using either fundamental analysis or technical analysis to make their recommendations.

The analyst has come across a company that they believe could be a good investment opportunity. The company is in the technology sector and has been growing rapidly over the past few years. The analyst has access to the following information:

1. Earnings per share (EPS) growth rate = 25%

2. Price-to-earnings (P/E) ratio = 20

3. Short interest = 10% of total shares outstanding

4. The company's stock price has been steadily rising for the past six months

Using the information available, which of the previous information will be considered Fundamental Analysis and which will be considered Technical Analysis?

Note: Use the numbering as an alias to the information.

Fundamental Analysis: 1,2

Technical Analysis: 3,4

1.3 

Explain the Fama-French factors.

Note: 1-3 sentences per factor.

Small Minus Big: Excess return of small-cap stocks over large-cap stocks.

High book-to-market Minus Low book-to-market: Return spread between value stocks (high book-to-market) and growth stocks (low book-to-market).

Conservative Minus Agressive: Return spread of companies with conservative investment strategies and those with aggressive investment strategies.

Robust Minus Weak: Return spread between companies with robust profitability and those with weak profitability.

Market Risk Premium: Excess return of the market over the risk-free rate. i.e., R_m - R_f

1.4 

Obtain the Return on Equity and Book to Market ratios for the most recent 5 observations in the Green-Hand-Zhang dataset for a selected stock of your choice.

Note: Post a screenshot of the whole data frame you get.

In [16]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd

In [18]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "rdittmar"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

In [29]:
df = pd.read_sql(
    '''
    SELECT date, ticker, bm, roeq, ret
    FROM data
    ORDER BY date, ticker;
    ''',
    conn
)

df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1018855 entries, 1 to 1022114
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   date    1018855 non-null  object 
 1   ticker  1018855 non-null  object 
 2   bm      1018855 non-null  float64
 3   roeq    1018855 non-null  float64
 4   ret     1018855 non-null  float64
dtypes: float64(3), object(2)
memory usage: 46.6+ MB


In [30]:
df[df.ticker=="AAPL"].tail()

,date,ticker,bm,roeq,ret
1010127,2021-11,AAPL,0.033233,0.356819,0.104940
1012546,2021-12,AAPL,0.033233,0.314320,0.074229
1014953,2022-01,AAPL,0.033233,0.314320,-0.015712
1017352,2022-02,AAPL,0.033233,0.314320,-0.054011
1019741,2022-03,AAPL,0.033233,0.319711,0.057473


1.5 

For this assignment, you will have to build a regression model that will predict returns based on ROE and BM.

Choose your favorite regression model

Train your model on data before December 31, 2021

Make predictions on data after December 31, 2021

Note: Upload a .ipynb notebook.

In [34]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [35]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

In [36]:
df = pd.read_sql(
    """
    select date, ticker, bm, roeq, ret
    from data
    order by date, ticker
    """,
    conn
  )
df = df.dropna()
df = df.set_index(["date", "ticker"])

In [37]:
later = df.index.get_level_values("date")>="2022-01"

train = df[~later]
test = df[later]

In [39]:
features = ["bm", "roeq"]

Xtrain = train[features]
ytrain = train["ret"]

model.fit(Xtrain, ytrain)

LinearRegression()

In [40]:
print("intercept =", model.intercept_)

coefs = pd.Series(model.coef_, index=features)
coefs

intercept = 0.011297834715994536


bm      0.000123
roeq    0.000022
dtype: float64

In [41]:
Xtest = test[features]

ypredict = model.predict(Xtest)
ypredict = pd.Series(ypredict, index=test.index)

In [42]:
cut = lambda x: pd.qcut(x, 5, labels=range(1, 6))
quintiles = ypredict.groupby("date", group_keys=False).apply(cut)
quintiles.name = "quintile"

In [43]:
test = test.join(quintiles)

rets = test.reset_index().groupby(["date", "quintile"]).ret.mean()
rets = rets.unstack()
print(rets.head())
print(rets.mean())

quintile         1         2         3         4         5
date                                                      
2022-01  -0.113110 -0.085502 -0.069989 -0.037387 -0.004984
2022-02   0.001906 -0.003371  0.015813  0.001610  0.018936
2022-03   0.017045  0.015988  0.009982  0.019922  0.020810
quintile
1   -0.031386
2   -0.024295
3   -0.014731
4   -0.005285
5    0.011587
dtype: float64


/var/folders/vy/755vvxhx3694gr0vrs_6xr6m0000gr/T/ipykernel_3063/438902182.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rets = test.reset_index().groupby(["date", "quintile"]).ret.mean()


1.6 

For this assignment, you will need to buy 10 stocks of your favorite stock and short 10 stock of your least favorite stock on your Alpaca paper account.

Note: Upload a .ipynb file.

In [44]:
!pip install alpaca-py

Defaulting to user installation because normal site-packages is not writeable


In [47]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

KEY = "your_key"
SECRET_KEY = "your_secrete_key"

trading_client = TradingClient(KEY, SECRET_KEY, paper=True)

In [48]:
# Test API connection
try:
    account = trading_client.get_account()
    print(f"Account status: {account.status}")
    print(f"Buying power: ${account.buying_power}")
except Exception as e:
    print(f"Error connecting to API: {e}")

Account status: ACTIVE
Buying power: $200000


In [49]:
order = MarketOrderRequest(
    symbol="NVDA",
    qty=10,
    side=OrderSide.BUY,
    time_in_force=TimeInForce.DAY
    )
_ = trading_client.submit_order(order_data=order)

In [50]:
order = MarketOrderRequest(
    symbol="TSLA",
    qty=10,
    side=OrderSide.SELL,
    time_in_force=TimeInForce.DAY
    )
_ = trading_client.submit_order(order_data=order)

In [51]:
positions = trading_client.get_all_positions()
positions

[]

In [52]:
account = trading_client.get_account()
account

{   'account_blocked': False,
    'account_number': 'PA3B3F2QFYTP',
    'accrued_fees': '0',
    'buying_power': '196211.85',
    'cash': '100000',
    'created_at': datetime.datetime(2025, 3, 16, 19, 4, 26, 467644, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100000',
    'id': UUID('120578b4-5e95-422e-aabd-fd7b1852582e'),
    'initial_margin': '1894.08',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '98105.92',
    'options_approved_level': 3,
    'options_buying_power': '96819.25',
    'options_trading_level': 3,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '100000',
    'regt_buying_power': '196211.85',
    'short_market_value': '0',
    'sho

In [54]:
account.equity

'100000'